In [1]:
# Import libraries
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import string
import spacy
import gensim
from gensim import corpora
import nltk
from nltk.corpus import stopwords
from gensim.models import LdaModel

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
# Replace with name of file with customer reviews
my_file_name = "Customer Feedback.xlsx"

In [3]:
# Load the csv file into the dataframe
review_data = pd.read_excel(my_file_name)
print(review_data.head(2))
print(len(review_data))

                  Date  Rating                 Type  \
0  2023-04-30 00:00:00       2   Investment I Claim   
1  2023-02-23 00:00:00       1   Investment I Renew   

                                              Review    Classification  
0  Your withdrawal process is getting slower by t...       Liquidation  
1                    Your team isn’t doing their job  Customer Service  
2185


In [4]:
# Make a copy of the original data to run it through my model FOR loop
original_data = pd.read_excel(my_file_name)
print(original_data['Review'][61])

Your App doesn't work in UK.  I am currently there and cannot use App to make my contributions.


In [5]:
# Function to clean the text
def clean_text(text, preserved_words=None):
    if preserved_words is None:
        preserved_words = []  # If no preserved_words list is provided, initialize an empty list
    
    # Tokenize the text
    text_tokens = text.split()
    
    # Combine tokens back into a cleaned sentence while preserving specific words
    cleaned_tokens = []
    for token in text_tokens:
        # Check if the token is a preserved word (ignoring case)
        if any(token.lower() == word.lower() for word in preserved_words):
            cleaned_tokens.append(token)
        else:
            # Filter out unwanted words based on length and digits
            if not (token.isdigit() or len(token) <= 3):
                # Remove punctuation from non-preserved words
                cleaned_token = token.translate(str.maketrans('', '', string.punctuation))
                if cleaned_token:  # Check if the cleaned token is not empty
                    cleaned_tokens.append(cleaned_token)
    
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text.lower()

In [6]:
# More preprocessing on the data
review_data.dropna(axis=0, how='any', inplace=True)

preserved_words = ['app']
review_data['Review'] = review_data['Review'].apply(clean_text, preserved_words=preserved_words)

print('-------Dataset-------')
print(review_data['Rating'].value_counts())
print(len(review_data))

print(review_data['Review'][61])

-------Dataset-------
Rating
3    903
1    661
2    564
4     40
5     17
Name: count, dtype: int64
2185
your app doesnt work currently there cannot app make contributions


In [7]:
# Make a copy of the review data
df_review = review_data.copy()

stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df_review['Review']=df_review['Review'].apply(remove_stopwords)

In [8]:
# Function for lemmatization
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

In [9]:
text_list = df_review['Review'].tolist()

# Tokenize the words and show sample
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

# Check how the tex_list looks
text_list[:5]

['team']


['withdrawal process getting slower day someone emergency request withdrawal anymore',
 'team isn’t',
 'system failure frequent wait hours portal downtime slow response hospital approval requests better please',
 'staff isobel rude attitude needs trained customer service',
 'settlement customers claim riddled forth back arguments accompanied stress']

In [10]:
# Create Matrix Dictionary
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

In [11]:
# We can view the words by using the dictionary
# View words in 10 documents in the matrix
id_words = [[(dictionary[id], count) for id, count in line] for line in doc_term_matrix[0:10]]
print(id_words)

[[('day', 1), ('emergency', 1), ('process', 1), ('request', 1), ('slow', 1), ('withdrawal', 2)], [('team', 1)], [('request', 1), ('slow', 1), ('approval', 1), ('downtime', 1), ('failure', 1), ('frequent', 1), ('hospital', 1), ('hour', 1), ('portal', 1), ('response', 1), ('system', 1), ('wait', 1)], [('attitude', 1), ('customer', 1), ('rude', 1), ('service', 1)], [('customer', 1), ('argument', 1), ('settlement', 1), ('stress', 1)], [('service', 1), ('poor', 1), ('yesterday', 1)], [('customer', 1), ('service', 1), ('client', 1), ('experience', 1), ('fast', 1), ('good', 1), ('guy', 1), ('guysnwill', 1), ('home', 1), ('leg', 1), ('men', 1), ('previous', 1), ('quarter', 1), ('think', 1), ('yearsmh', 1)], [('service', 2), ('poor', 1), ('platform', 1), ('prompt', 1)], [('service', 1)], [('request', 1), ('response', 1), ('poor', 1), ('account', 1), ('clientservice', 1), ('email', 1), ('last', 1), ('mail', 1), ('staff', 1), ('week', 1)]]


In [12]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model with 20 topics
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=20, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [13]:
# Dump and load model
saved_lda = "lda_model_v2"
lda_model.save(saved_lda)

# Load the saved LDA model
lda_model_load = LdaModel.load(saved_lda)

In [14]:
# Display model Topics
lda_model.print_topics()

[(0,
  '0.071*"axamansard" + 0.049*"time" + 0.041*"satisfied" + 0.033*"emergency" + 0.031*"card" + 0.031*"high" + 0.030*"sick" + 0.028*"problem" + 0.026*"surgery" + 0.017*"timely"'),
 (1,
  '0.159*"service" + 0.109*"poor" + 0.085*"customer" + 0.046*"liquidation" + 0.043*"week" + 0.035*"response" + 0.029*"time" + 0.026*"request" + 0.023*"email" + 0.019*"complaint"'),
 (2,
  '0.095*"service" + 0.074*"year" + 0.067*"good" + 0.058*"well" + 0.047*"improvement" + 0.040*"part" + 0.039*"clinic" + 0.028*"customer" + 0.027*"late" + 0.021*"last"'),
 (3,
  '0.139*"interest" + 0.095*"rate" + 0.032*"choice" + 0.031*"benefit" + 0.026*"daily" + 0.022*"nit" + 0.021*"return" + 0.020*"year" + 0.020*"instance" + 0.020*"current"'),
 (4,
  '0.108*"payment" + 0.080*"policy" + 0.038*"case" + 0.037*"officer" + 0.023*"urgent" + 0.018*"agent" + 0.018*"insurance" + 0.017*"network" + 0.016*"message" + 0.016*"complete"'),
 (5,
  '0.072*"confirmation" + 0.069*"hospital" + 0.059*"call" + 0.037*"feedback" + 0.036*"che

In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
#vis
pyLDAvis.display(vis)

In [16]:
# Metrics for the lda_model
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.26258348938944

Coherence Score:  0.3954538065421478


## LAYERED SVM 

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

In [18]:
topic_probs = []
for doc_bow in doc_term_matrix:
    doc_topics = lda_model.get_document_topics(doc_bow)
    probs = [prob for _, prob in doc_topics]
    topic_probs.append(probs)

In [19]:
# Length of topic_probs is num of texts in a document
topic_probs[20]

[0.11633754, 0.17603818, 0.08750296, 0.45587328, 0.10174422]

In [20]:
print(len(topic_probs))
print(len(df_review.Classification))

2185
2185


In [21]:
# Convert the topic probabilities and labels to numpy arrays
X = pd.DataFrame(topic_probs).values
X[np.isnan(X)] = 0  # Replace NaN values with zero
y = df_review['Classification'].values

In [22]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the SVM classifier
svm = SVC(kernel='linear', C=1.0)
svm.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm.predict(X_test)

#### Add MOT as a feature for the SVM model

In [24]:
from sklearn.preprocessing import LabelEncoder

# Convert the topic probabilities to a DataFrame with string column names
X_lda = pd.DataFrame(topic_probs).values
X_lda[np.isnan(X_lda)] = 0  # Replace NaN values with zero

# Encode the categorical feature 'category' using label encoding
label_encoder = LabelEncoder()
df_review['Type_encoded'] = label_encoder.fit_transform(df_review['Type'])
# Convert the encoded categorical feature to a DataFrame
X_category_encoded = pd.DataFrame(df_review['Type_encoded'], columns=['Type_encoded'])

# Combine LDA features and the encoded categorical feature
X = pd.concat([pd.DataFrame(X_lda), X_category_encoded], axis=1)
# Convert column names to strings
X.columns = X.columns.astype(str)

# Set the target variable
y = df_review['Classification'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the SVM classifier
svm = SVC(kernel='linear', C=1.0)
svm.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm.predict(X_test)

# Evaluate the SVM classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation results
print('Accuracy:', accuracy)
print('Classification Report:')
print(report)

Accuracy: 0.32265446224256294
Classification Report:
                                       precision    recall  f1-score   support

                        Approval Time       0.33      0.92      0.49       116
                    Claims Adjustment       0.00      0.00      0.00         4
                       Claims Payment       0.00      0.00      0.00         3
                     Customer Service       1.00      0.03      0.05        38
                        Drug Delivery       0.00      0.00      0.00         2
                        Enrollee Card       0.00      0.00      0.00         4
                  Gym/Wellness Access       0.00      0.00      0.00         3
                         Hospital QoS       0.38      0.10      0.15        31
Hospital/Pharmacy/Telemedicine Access       0.00      0.00      0.00         3
                       Incorrect Data       0.00      0.00      0.00         1
                        Inflow Update       0.00      0.00      0.00        2

c:\Users\bmelu-akekue\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bmelu-akekue\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bmelu-akekue\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

In [25]:
# save the svm model
import joblib
model_filename = "svm_model_v2.pkl"
joblib.dump(svm, model_filename)

['svm_model_v2.pkl']

In [26]:
# Load the SVM model
svm_model_loaded = joblib.load('svm_model_v2.pkl')

In [27]:
# Using this to test code below
test_sample = df_review[['Review', 'Type']][:400]

# Data used to check accuracy against predicted labels
correct_preds = df_review[['Review', 'Classification']][:400]

In [28]:
# Function to preprocess the new reviews and get LDA topic probabilities
def preprocess_new_reviews(new_reviews):
    # Make use of clean_text, remove_stopwords
    new_reviews.Review.apply(clean_text, preserved_words=preserved_words)
    new_reviews.Review.apply(remove_stopwords)
    listed = new_reviews['Review'].tolist()
    tokenized_new_reviews = lemmatization(listed)
    dictionary = corpora.Dictionary(tokenized_new_reviews)

    # Create a bag-of-words corpus for the new_reviews
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_new_reviews]

    # Get LDA topic probabilities for the new_reviews
    topic_probs = []
    for doc_bow in corpus:
        doc_topics = lda_model.get_document_topics(doc_bow)
        probs = [prob for _, prob in doc_topics]
        topic_probs.append(probs)
    
    X_data = pd.DataFrame(topic_probs).values
    X_data[np.isnan(X_data)] = 0  # Replace NaN values with zero

    return X_data

In [29]:
# Function to classify new reviews using SVM model
def classify_new_reviews(new_reviews_lda_probs, new_reviews_type):
    # Combine LDA features (new_reviews_lda_probs) with encoded 'Type' feature
    X_combined_features = pd.concat([pd.DataFrame(new_reviews_lda_probs), new_reviews_type], axis=1)
    # Convert column names to strings
    X_combined_features.columns = X_combined_features.columns.astype(str)
    # Replace NaN values with zero
    X_combined_features.fillna(0, inplace=True)

    # Make predictions using the SVM model
    predictions = svm.predict(X_combined_features)
    
    return predictions

In [30]:
new_reviews = test_sample

# Preprocess the new reviews and get LDA topic probabilities
lda_probs_new_reviews = preprocess_new_reviews(new_reviews)
print("Shape of x_data:", new_reviews.shape)
lda_probs_new_reviews[np.isnan(lda_probs_new_reviews)] = 0  # Replace NaN values with zero

# Sample new 'Type' values for the new reviews (replace this with actual data)
new_reviews_type = new_reviews.Type

# Encode the 'Type' feature using LabelEncoder
encoded_new_reviews_type = label_encoder.transform(new_reviews_type)
new_reviews_type_df = pd.DataFrame(encoded_new_reviews_type, columns=['Type_encoded'])

# Classify the new reviews using SVM model
predicted_labels = classify_new_reviews(lda_probs_new_reviews, new_reviews_type_df)

# Print the results
results_df = pd.DataFrame({
    'Review': new_reviews['Review'],
    'Type': new_reviews['Type'],
    'Predicted Label': predicted_labels
})
print(results_df)

# # Read to excel file
# file_path = 'output.xlsx'
# results_df.to_excel(file_path, index=False)

Shape of x_data: (400, 2)
                                                Review                 Type  \
0    withdrawal process getting slower day someone ...   Investment I Claim   
1                                           team isn’t   Investment I Renew   
2    system failure frequent wait hours portal down...       Health I Claim   
3    staff isobel rude attitude needs trained custo...       Health I Claim   
4    settlement customers claim riddled forth back ...        Motor I Claim   
..                                                 ...                  ...   
395                                      response slow       Health I Claim   
396                            response team slow poor       Health I Claim   
397              redemption took long used much faster   Investment I Claim   
398  recently response timing terrible coupled inve...         Health I Buy   
399  recently liquidated account asking took foreve...   Investment I Claim   

      Predicted Label  
0

In [35]:
# ACCURACY OF MODEL WITHOUT POST-PROCESSING

# Combine the predicted labels and the original reviews data
check_df = pd.DataFrame({
    'Review': df_review['Review'][:400], # I have to use preprocessed, unless data won't merge correctly
    'Predicted Label': predicted_labels
})

# Merge with the corrected predicted labels to get the actual labels for comparison
check_df = check_df.merge(correct_preds, on='Review', suffixes=('_predicted', '_corrected'))

# Calculate the accuracy
accuracy = (check_df['Predicted Label'] == check_df['Classification']).mean()

print('Accuracy:', accuracy)

# Save the results to an Excel file
check_df.to_excel('no_heuristics.xlsx', index=False)

Accuracy: 0.36428571428571427


#### ADDING POST-PROCESSING TO SEE HOW IT AFFECTS THE ACCURACY

In [32]:
# Trying to Hard-code some words to some labels ## Rule-based post-processing OR heuristics-based correction
def adjust_predicted_label(review_text, predicted_label):
    # Define keywords and their associated new labels
    keyword_labels = {
        'Technological Issues': {'app', 'software', 'update', 'bug', 'crash', 'error', 'glitch', 'slow', 'lag', 'freeze', 'dashboard', 'application', 'chatbot', 'website'},
        'Claim Adjustment': {'settlement', 'claims', 'claim'},
        'Delayed Liquidation': {'withdrawal', 'redemption', 'account', 'liquidation', 'investment'},
        'Policy Cover': {'bill', 'coverage', 'plan', 'HMO'},
        'Approval Delay': {'approval', 'confirmation'},
        'Interest Rate': {'interest', 'rate'},
        'Hospital QoS': {'hospital', 'clinics', 'doctor', 'doctors', 'drugs', 'drug', 'admit', 'dental', 'healthcare', 'care', 'prescription', 'prescribe'},
        'Gym/Wellness Access': {'gym', 'fitness'},
        'Slow Onboarding': {'plan', 'onboard'},
        'Enrollee Card Delay': {'card'},
        'Delayed Inflow Update': {'inflow'}, # New ones are under
        
        # Add more labels and their corresponding sets of keywords as needed
    }

    # Tokenize the review text
    review_tokens = review_text.lower().split() # Converts all to lowercase

    # Check if any keyword is present in the review text
    for label, keywords in keyword_labels.items():
        if any(keyword in review_tokens for keyword in keywords):
            return label

    # If no keyword is found, return the original predicted label
    return predicted_label

In [33]:
# Create an empty list to store the adjusted labels
adjusted_labels = []

review_texts = test_sample.Review

# Iterate through each review text and predicted label
for review_text, predicted_label in zip(review_texts, predicted_labels):
    # Apply the adjust_predicted_label function to get the adjusted label
    adjusted_label = adjust_predicted_label(review_text, predicted_label)
    
    # Append the adjusted label to the list
    adjusted_labels.append(adjusted_label)

# Convert the list of adjusted labels back to a pandas Series
adjusted_labels_series = pd.Series(adjusted_labels)

# Create a DataFrame to store the data
data_adj = pd.DataFrame({
    'Review Text': review_texts,
    'Predicted Labels': predicted_labels,
    'Adjusted Labels': adjusted_labels_series
})

# Save the DataFrame to an Excel file
#data_adj.to_excel('adjusted.xlsx', index=False)

In [34]:
# Compare Accuracy of adjusted label against original labels (With Heuristics)
# Combine the predicted labels and the original reviews data
check_adj = pd.DataFrame({
    'Review': df_review['Review'][:400], # I have to use preprocessed, unless data won't merge correctly
    'Actual Labels': df_review['Classification'][:400],
    'Adjusted Labels': adjusted_labels_series,
})

# Calculate the accuracy
accuracy = (check_adj['Actual Labels'] == check_adj['Adjusted Labels']).mean()

print('Accuracy:', accuracy)

# Save the results to an Excel file
#check_adj.to_excel('check_adj.xlsx', index=False)

Accuracy: 0.2075
